In [ ]:
from ipywidgets import widgets, interact, interactive, IntSlider
from IPython.display import display
import dicom
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.ndimage
import matplotlib.pyplot as plt
import os
scan_path = os.path.join(os.getcwd(),'stage1')
patients = sorted(os.listdir(scan_path))

In [ ]:
patient_list = widgets.Select(description = 'Patients',options = patients)
image_slider = IntSlider(description = "Image Number", min = 0)

patient_path = os.path.join(scan_path,patient_list.value)
image_list = os.listdir(patient_path)
num_scans = len(os.listdir(patient_path))
image_slider.max = num_scans

img_pixels = dicom.read_file(os.path.join(patient_path,image_list[image_slider.value])).pixel_array
my_img = plt.imshow(img_pixels,cmap=plt.cm.gray)
plt.show()

def update_image_slider(args):
    patient_path = os.path.join(scan_path,args['new'])
    num_scans = len(os.listdir(patient_path))
    image_list = os.listdir(patient_path)
    image_slider.max = num_scans
    
def view_image(args):
    img = dicom.read_file(os.path.join(patient_path,image_list[int(args['new'])])) 
    img_pixels = img.pixel_array
    my_img = plt.imshow(img_pixels,cmap=plt.cm.gray)
    plt.show()
    
    
patient_list.observe(update_image_slider,'value')
image_slider.observe(view_image,'value')

display(patient_list)
display(image_slider)